In [174]:
import numpy as np
import math
import json
import glob
from functools import reduce

In [332]:
def load_results(dataset, model, metric, ref, n_sample, seed):
    result_dir = f'../results/{dataset}/{model}_{seed}/{metric}_{ref}_{n_sample}'
      
    result_fs = glob.glob(f'{result_dir}/*.json')
    results = []
    for file in result_fs:
        with open(file, 'r') as f:
            data = json.load(f)
            print(file, len(data))
        results.extend(data)
        
    return results

In [577]:
metric = 'EMD'
dataset = 'CIFAR-100'
arch = 'resnet18'
n_sample = 10000
ref = 'val'
seed = '1'
logits_results = load_results(dataset, arch, metric, ref, n_sample, seed)

../results/CIFAR-100/resnet18_1/EMD_val_10000/clean.json 1
../results/CIFAR-100/resnet18_1/EMD_val_10000/brightness.json 5
../results/CIFAR-100/resnet18_1/EMD_val_10000/defocus_blur.json 5
../results/CIFAR-100/resnet18_1/EMD_val_10000/elastic_transform.json 5
../results/CIFAR-100/resnet18_1/EMD_val_10000/fog.json 5
../results/CIFAR-100/resnet18_1/EMD_val_10000/frost.json 5
../results/CIFAR-100/resnet18_1/EMD_val_10000/gaussian_blur.json 5
../results/CIFAR-100/resnet18_1/EMD_val_10000/gaussian_noise.json 5
../results/CIFAR-100/resnet18_1/EMD_val_10000/glass_blur.json 5
../results/CIFAR-100/resnet18_1/EMD_val_10000/impulse_noise.json 5
../results/CIFAR-100/resnet18_1/EMD_val_10000/jpeg_compression.json 5
../results/CIFAR-100/resnet18_1/EMD_val_10000/motion_blur.json 5
../results/CIFAR-100/resnet18_1/EMD_val_10000/pixelate.json 5
../results/CIFAR-100/resnet18_1/EMD_val_10000/saturate.json 5
../results/CIFAR-100/resnet18_1/EMD_val_10000/shot_noise.json 4


In [578]:
len(logits_results)

70

In [579]:
import altair as alt
import pandas as pd

In [580]:
corr = alt.Chart(alt.Data(values=logits_results)).mark_point().encode(
    x=alt.X('metric:Q', title=metric),
    y='error:Q',
    color=alt.Color('corruption:N', scale=alt.Scale(scheme='category20')),
    shape='corruption level:O'
).properties(
    width=250,
    height=250
)

line = pd.DataFrame({'metric': [0, 1], 'error': [0, 1]})
line_plot = alt.Chart(line).mark_line(color='black', strokeDash=[5, 8]).encode(
    x='metric',
    y='error',
)

if metric in ['ConfScore', 'ATC', 'AgreeScore', 'EMD']:
    plt = corr + line_plot

elif metric == 'Entropy':
    plt = corr

plt


alt.LayerChart(...)

In [581]:
def polyfit(x, y, degree=1):
    results = {}

    coeffs = np.polyfit(x, y, degree)

    results['polynomial'] = coeffs.tolist()

    p = np.poly1d(coeffs)

    yhat = p(x)                
    ybar = np.sum(y)/len(y)          
    ssreg = np.sum((yhat - ybar)**2)   
    sstot = np.sum((y - ybar)**2)    
    results['determination'] = ssreg / sstot

    return results

In [582]:
from scipy.stats import spearmanr
import math

In [583]:
d = [i['metric'] for i in logits_results]
e = [i['error'] for i in logits_results]
print(f'{metric} r2:', polyfit(d, e)['determination'])

coef, p = spearmanr(d, e)
print(f'{metric} rho:', coef)

print(f'{metric} slope:', polyfit(d, e)['polynomial'][0])
print(f'{metric} bias:', polyfit(d, e)['polynomial'][1])

EMD r2: 0.9940046620308471
EMD rho: 0.9955909369259032
EMD slope: 1.109920377530782
EMD bias: -0.023300986875067837


In [584]:
yhat = np.array(d)
y = np.array(e)

print('MAE:', np.abs(yhat - y).mean() * 100)

MAE: 2.9957093300522835
